Artigos pra ler: Student behaviour in unsuporvised online quizzzes : a closer look

In [1]:
import re
import os
import math
import pandas as pd
import datetime

In [2]:
path = 'Dataset/2018-1/'

cat database/2432/399_1326.log

In [3]:
dictHorarios = {}
colunasTabela = {'Aluno':['00'],
    'Laboratorio':['00'],
    'Exercicio':['00'],
    'Horario inicial':['00'],
    'Horario final':['00'],
    'Tempo decorrido':['00']
}

df2 = pd.DataFrame(columns=['oi','sa'])
print(df2)




Empty DataFrame
Columns: [oi, sa]
Index: []


In [4]:
#Coleta a data inicial e final de cada exercicio junto com o seu horario
def coletaDatas(arquivo):
    horarios = []
    hBegin = -1
    hEnd = -1
    dataIni = -1
    dataFim = -1
    horaIni = -1
    horaFim = -1
    for linha in arquivo:
        busca = re.findall(r'\d{4}-\d{1,2}-\d{1,2} \d+:\d+:\d+\.\d+',linha)
        if(len(busca) > 0):
            horaIni = busca[0]
            horarios.append(horaIni)
    if(len(horarios)>0):
        hBegin = horarios[0]
        hEnd = horarios[-1]
    if(hBegin != -1):
        busquinha = re.findall(r'\d{4}-\d{1,2}-\d{1,2}',hBegin)
        if(len(busquinha) > 0):
            dataIni = busquinha[0]
        busquinha = re.findall(r'\d+:\d+:\d+\.\d+',hBegin)
        if(len(busquinha) > 0):
            horaIni = busquinha[0]
    if(hEnd != -1):
            busquinha = re.findall(r'\d{4}-\d{1,2}-\d{1,2}',hEnd)
            if(len(busquinha)>0):
                    dataFim = busquinha[0]
            busquinha = re.findall(r'\d+:\d+:\d+\.\d+',hEnd)
            if(len(busquinha) > 0):
                horaFim = busquinha[0]
    '''            
    print("data raw:")
    print(hBegin,hEnd)
    print(dataIni, dataFim)
    print("Datas")
    print(horaIni,horaFim)
    print("EOF")
    '''
    return horarios, dataIni, dataFim, horaIni, horaFim

In [5]:
#Calcula a quantidade de linhas de um arquivo de log
def qntd_linhas_log(arquivo):
    contador = 0
    paste = 0
    delete = 0
    for linha in arquivo.split("\n"):
        contador += 1
        busca = re.search(r'paste',linha)
        busca2 = re.search(r'delete',linha)
        busca3 = re.search(r'Backspace',linha)
        if(busca): paste += 1
        if(busca2): delete += 1
        if(busca3): delete += 1
    
    p1 = paste/contador
    p2 = delete/contador
    return contador,round(p1,2),round(p2,2)
    

In [6]:
#Calcula o tempo empenhado no exercicio
def tempoEmpenhado(data_ini, data_fim, hora_ini, hora_fim):
    s = '' + str(data_ini) + ' ' + str(hora_ini) 
    r = '' + str(data_fim) + ' ' +str(hora_fim)
    date1 = int(datetime.datetime.strptime(s, '%Y-%m-%d %H:%M:%S.%f').strftime("%s"))
    date2 = int(datetime.datetime.strptime(r, '%Y-%m-%d %H:%M:%S.%f').strftime("%s"))
    difdate = date2 - date1
    return difdate

In [7]:
#Calcula a quantidade de dias em que o mesmo exercicio foi resolvido
def n_acessosColetor(arquivo):
    primeiroAcesso = 1
    difdate = 0
    for linha in arquivo:
        busca = re.search(r'\d{4}-\d{1,2}-\d{1,2}',linha)
        if(busca):
            if(primeiroAcesso):
                dataIni = busca.group(0)
                primeiroAcesso = 0
            dataAtual = busca.group(0)
            #date1 = int(datetime.datetime.strptime(dataIni, '%Y-%m-%d').strftime("%d"))
            #date2 = int(datetime.datetime.strptime(dataAtual, '%Y-%m-%d').strftime("%d"))
            #difdate = date2 - date1
            start_date = datetime.datetime.strptime(dataIni, '%Y-%m-%d')
            end_date = datetime.datetime.strptime(dataAtual, '%Y-%m-%d')
            difdate = abs(end_date-start_date)
            difdate = difdate.days
            

            
    
    return difdate

Funcoes na pasta Executions

In [8]:
#Ve a quantidade de submissoes em um exercicio
def attemptsColetor(arquivo):
    contador = 0
    for linha in arquivo.split("\n"):
        busca = re.search(r'SUBMITION',linha)
        if(busca): contador += 1
    return contador

In [9]:
#Ve a quantidade de testes em um exercicio
def testColetor(arquivo):
    contador = 0
    for linha in arquivo.split("\n"):
        busca = re.search(r'TEST',linha)
        if(busca): contador += 1
    return contador

Funcoes na pasta Scores

In [10]:
#Ve a quantidade de exercicios em um laboratorio
def exercicios_no_lab(arquivo):
    qntd = 0
    for linha in arquivo.split("\n"):
        busca = re.search(r'number of exercises: \d+',linha)
        if(busca): 
            busca2 = re.search(r'\d+',busca.group(0))
            return busca2.group(0)

In [11]:
#Ve a quantidade de exercicios respondidos corretamente em um laboratorio
def exer_certos_no_lab(arquivo):
    qntd = 0
    for linha in arquivo.split("\n"):
        busca = re.search(r'correct: \d+',linha)
        if(busca): 
            busca2 = re.search(r'\d+',busca.group(0))
            return busca2.group(0)

In [12]:
#Nota final por laboratorio
def nota_lab(arquivo):
    qntd = 0
    for linha in arquivo.split("\n"):
        busca = re.search(r'final score  \(0-10\): \d+\.\d+',linha)
        if(busca):
            busca2 = re.search(r'\d+\.\d+',busca.group(0))
            return busca2.group(0)

In [13]:
#Quantidade de linhas de um codigo
def qntd_linhas(arquivo):
    qntd = 0
    for linha in arquivo.split("\n"):
        busca = re.search(r'\w*',linha)
        if(busca): qntd += 1
    return qntd

In [14]:
#Quantidade de linhas de um codigo sem comentarios etc
def qntd_linhas_filtrado(arquivo):
    qntd = 0
    for linha in arquivo.split("\n"):
        busca = re.search(r'\S',linha)
        if(busca): 
            busca2 = re.search(r'^(?!#).*',busca.group(0))
            if(busca2): qntd += 1

    return qntd

In [15]:
def keystroke_latency(arquivo):
    for linha in arquivo.split("\n"):
        hora = re.search(r'\d{4}-\d{1,2}-\d{1,2}',linha)

In [16]:
#Definindo o DataFrame
colunas = ['Aluno','Lab','Exercicio','Duracao','Qntd_Linhas','Qntd_Linhas_Filtrado','Testes','Submissoes','N_Dias','PP_Paste','PP_Delete','Linhas_Log','Nota']
df = pd.DataFrame(columns = colunas)
print(df)
df = df[0:0]

Empty DataFrame
Columns: [Aluno, Lab, Exercicio, Duracao, Qntd_Linhas, Qntd_Linhas_Filtrado, Testes, Submissoes, N_Dias, PP_Paste, PP_Delete, Linhas_Log, Nota]
Index: []


In [17]:
#MAIN
for folder in os.listdir(path):
    if not folder.startswith('.'):
        
        directory = os.path.join(path,folder)+'/users/'
        classe = folder
        for aux in os.listdir(directory):
            print(directory,aux)
            #directory = os.path.join(directory,aux)
            #pastaLogs = directory + aux + '/codemirror'
            pastaLogs = os.path.join(directory,aux) + '/codemirror'
            #Trabalhando com o arquivo do logs de interacao
            aluno = aux
            for file in os.listdir(pastaLogs):
                buscaL = re.findall(r'\d+',file)
                if(len(buscaL) > 0):
                    laboratorio = buscaL[0]
                    exercicio = buscaL[1]
                file = os.path.join(pastaLogs,file)
                arquivo = open(file,'r',encoding='ISO-8859-1')
                data = arquivo.read()
                dataSplitado = data.split("\n")
                #-/-
                #parte para coleta de horario
                vetHorarios, date_begin, date_end, hour_begin, hour_end = coletaDatas(dataSplitado)
                dictHorarios[str(exercicio)] = vetHorarios

                if(hour_begin != -1):
                    #calcula o tempo emepnhado
                    exr_duration = tempoEmpenhado(date_begin, date_end, hour_begin, hour_end)
                #Coleta numero de acessos
                n_acessos = n_acessosColetor(dataSplitado)
                if(n_acessos == 0): n_acessos = 1
                qt_linhas, pp_paste, pp_delete = qntd_linhas_log(data)
                df = df.append({'Aluno': aluno,'Lab' : laboratorio,'Exercicio' : exercicio,'Duracao':exr_duration
                                ,'Qntd_Linhas':0,'Qntd_Linhas_Filtrado':0,'Testes':0,'Submissoes':0,'N_Dias':n_acessos,
                                'PP_Paste':pp_paste,'PP_Delete':pp_delete,'Linhas_Log':qt_linhas,'Nota':0.0}, ignore_index=True)    


            #a partir de agora eu trabalho com outro tipo de arquivo para a coleta, o de tests e submissions
            pastaExecucoes = os.path.join(directory,aux) + '/executions'
            for file in os.listdir(pastaExecucoes):
                buscaL = re.findall(r'\d+',file)
                if(len(buscaL) > 0):
                    laboratorio = buscaL[0]
                    exercicio = buscaL[1]  
                file = os.path.join(pastaExecucoes,file)
                arquivo = open(file,'r',encoding='ISO-8859-1')
                data = arquivo.read()
                #Quantidade de submissoes por exercicio
                attempts = attemptsColetor(data)

                #Quantidade de testes por exercicio
                tests = testColetor(data)
                df.loc[(df['Aluno'] == aluno) & (df['Lab'] == laboratorio) & (df['Exercicio'] == exercicio)
                       ,'Testes'] = tests
                df.loc[(df['Aluno'] == aluno) & (df['Lab'] == laboratorio) & (df['Exercicio'] == exercicio)
                       ,'Submissoes'] = attempts

            #Coletando atributos da pasta score 
            pastaScores = os.path.join(directory,aux) + '/grades'
            for file in os.listdir(pastaScores):
                buscaL = re.findall(r'\d+',file)
                if(len(buscaL) > 0):
                    laboratorio = buscaL[0]  
                #file = os.path.join(pastaScores,file)
                #arquivo = open(file,'r',encoding='ISO-8859-1')
                #data = arquivo.read()
                #Quantidade de exercicios do laboratorio
                #qntd_exercicios = exercicios_no_lab(data)
                #Quantidade de exercicios do laboratorio respondidos corretamente
                #qntd_exer_corretos = exer_certos_no_lab(data)
                #Nota do lab
                #nota_do_lab = nota_lab((data))
                #if(nota_do_lab == None): nota_do_lab = 0
                #df.loc[(df['Aluno'] == aluno) & (df['Lab'] == laboratorio),'Nota'] = float(nota_do_lab)
                df.loc[(df['Aluno'] == aluno) & (df['Lab'] == laboratorio),'Nota'] = 0

            #Coletando atributos da pasta codes
            pastaCodigos = os.path.join(directory,aux) + '/codes'
            for file in os.listdir(pastaCodigos):
                buscaL = re.findall(r'\d+',file)
                if(len(buscaL) > 0):
                    laboratorio = buscaL[0]
                    exercicio = buscaL[1]  
                file = os.path.join(pastaCodigos,file)
                arquivo = open(file,'r',encoding='ISO-8859-1')
                data = arquivo.read()
                #Quantidade de linhas de um codigo
                linhas_codigo = qntd_linhas(data)
                #Quantidade de linhas de um codigo sem comentarios, importacoes
                linhas_codigo_filtrado = qntd_linhas_filtrado(data)
                df.loc[(df['Aluno'] == aluno) & (df['Lab'] == laboratorio) & (df['Exercicio'] == exercicio)
                       ,'Qntd_Linhas'] = linhas_codigo
                df.loc[(df['Aluno'] == aluno) & (df['Lab'] == laboratorio) & (df['Exercicio'] == exercicio)
                       ,'Qntd_Linhas_Filtrado'] = linhas_codigo_filtrado

            


Dataset/2018-1/180/users/ 1168
Dataset/2018-1/180/users/ 1502
Dataset/2018-1/180/users/ 1591
Dataset/2018-1/180/users/ 2182
Dataset/2018-1/180/users/ 2306
Dataset/2018-1/180/users/ 2495
Dataset/2018-1/180/users/ 2535
Dataset/2018-1/180/users/ 2613
Dataset/2018-1/180/users/ 2639
Dataset/2018-1/180/users/ 2869
Dataset/2018-1/180/users/ 3031
Dataset/2018-1/180/users/ 3222
Dataset/2018-1/180/users/ 3223
Dataset/2018-1/180/users/ 3224
Dataset/2018-1/180/users/ 3225
Dataset/2018-1/180/users/ 3226
Dataset/2018-1/180/users/ 3227
Dataset/2018-1/180/users/ 3228
Dataset/2018-1/180/users/ 3229
Dataset/2018-1/180/users/ 3230
Dataset/2018-1/180/users/ 3231
Dataset/2018-1/180/users/ 3232
Dataset/2018-1/180/users/ 3233
Dataset/2018-1/180/users/ 3234
Dataset/2018-1/180/users/ 3235
Dataset/2018-1/180/users/ 3236
Dataset/2018-1/180/users/ 3237
Dataset/2018-1/180/users/ 3238
Dataset/2018-1/180/users/ 3239
Dataset/2018-1/180/users/ 3240
Dataset/2018-1/180/users/ 3241
Dataset/2018-1/180/users/ 3242
Dataset/

Dataset/2018-1/186/users/ 3153
Dataset/2018-1/186/users/ 3257
Dataset/2018-1/186/users/ 3263
Dataset/2018-1/186/users/ 3264
Dataset/2018-1/186/users/ 3266
Dataset/2018-1/186/users/ 3302
Dataset/2018-1/186/users/ 3451
Dataset/2018-1/186/users/ 3456
Dataset/2018-1/186/users/ 3495
Dataset/2018-1/186/users/ 3506
Dataset/2018-1/186/users/ 3570
Dataset/2018-1/186/users/ 3642
Dataset/2018-1/187/users/ 1543
Dataset/2018-1/187/users/ 1913
Dataset/2018-1/187/users/ 2300
Dataset/2018-1/187/users/ 2409
Dataset/2018-1/187/users/ 2445
Dataset/2018-1/187/users/ 2535
Dataset/2018-1/187/users/ 2978
Dataset/2018-1/187/users/ 3145
Dataset/2018-1/187/users/ 3146
Dataset/2018-1/187/users/ 3147
Dataset/2018-1/187/users/ 3148
Dataset/2018-1/187/users/ 3149
Dataset/2018-1/187/users/ 3150
Dataset/2018-1/187/users/ 3151
Dataset/2018-1/187/users/ 3153
Dataset/2018-1/187/users/ 3155
Dataset/2018-1/187/users/ 3188
Dataset/2018-1/187/users/ 3190
Dataset/2018-1/187/users/ 3191
Dataset/2018-1/187/users/ 3192
Dataset/

In [18]:
print("aluno:",aluno)
print("Dia inicio:",date_begin, "Dia fim:",date_end, "Hora inicio:",hour_begin,"Hora fim:", hour_end)
print("Numero de dias empenhados durante exercicio:",n_acessos)
print("Tempo empenhado na atividade em segundos:",exr_duration)
print("Numero do lab:",laboratorio,"exercicio:",exercicio,"submissoes:",attempts,"testes:",tests)
print("Total de exercicios:",qntd_exercicios,"Total respondido:",qntd_exer_corretos,"nota:",nota_do_lab)
print("Quantidade de linhas de codigo:",linhas_codigo, "Quantidade de linhas de codigo sem comentarios/importacoes:",linhas_codigo_filtrado)

aluno: 3569
Dia inicio: 2018-4-17 Dia fim: 2018-4-17 Hora inicio: 10:46:13.287 Hora fim: 10:47:17.947
Numero de dias empenhados durante exercicio: 1
Tempo empenhado na atividade em segundos: 64
Numero do lab: 1030 exercicio: 835 submissoes: 0 testes: 1


NameError: name 'qntd_exercicios' is not defined

In [36]:
df['Aluno'] = df['Aluno'].astype(int)
df['Lab'] = df['Lab'].astype(int)
df['Duracao'] = df['Duracao'].astype(int)
df['Qntd_Linhas'] = df['Qntd_Linhas'].astype(int)
df['Qntd_Linhas_Filtrado'] = df['Qntd_Linhas_Filtrado'].astype(int)
df['Testes'] = df['Testes'].astype(int)
df['Submissoes'] = df['Submissoes'].astype(int)
df['N_Dias'] = df['N_Dias'].astype(int)
df['PP_Paste'] = df['PP_Paste'].astype(float)
df['PP_Delete'] = df['PP_Delete'].astype(float)
df['Linhas_Log'] = df['Linhas_Log'].astype(int)
df['Nota'] = df['Nota'].astype(float) 

In [37]:
#Porcentagem de exercicios corretos/ total execicios por lab



In [38]:
agrupado =  df.groupby(['Aluno','Lab'],as_index=False).agg({'Duracao': 'sum', 'Qntd_Linhas': 'sum', 'Qntd_Linhas_Filtrado': 'sum'
                                             , 'Testes': 'sum', 'Submissoes': 'sum', 'PP_Paste': 'sum'
                                             , 'PP_Delete': 'sum', 'Linhas_Log': 'sum', 'Nota':'mean'})

In [39]:
agrupado

,Aluno,Lab,PP_Delete,Duracao,PP_Paste,Qntd_Linhas,Linhas_Log,Qntd_Linhas_Filtrado,Nota,Submissoes,Testes
0,1085,1031,0.53,80,0.00,25,19,9,0.0,0,1
1,1085,1083,0.34,296,0.01,24,372,11,0.0,4,12
2,1085,1118,0.69,3825,0.01,112,3484,80,0.0,23,82
3,1085,1265,0.30,1864,0.00,32,991,20,0.0,17,42
4,1163,1012,3.03,772009,0.10,35,1159,29,0.0,34,17
5,1163,1029,2.54,5881135,0.04,128,4996,74,0.0,34,77
6,1163,1051,2.73,616820,0.05,109,5529,82,0.0,32,158
7,1163,1052,0.47,1089,0.01,25,900,17,0.0,3,9
8,1163,1084,2.04,1665611,0.12,145,4158,86,0.0,69,133
9,1163,1089,0.14,409784,0.55,73,80,50,0.0,8,20


In [40]:
agrupadoMedia = df.groupby(['Aluno','Lab'],as_index=False).agg({'Duracao': 'mean', 'Qntd_Linhas': 'mean', 'Qntd_Linhas_Filtrado': 'mean'
                                             , 'Testes': 'mean', 'Submissoes': 'mean', 'PP_Paste': 'mean'
                                             , 'PP_Delete': 'mean', 'Linhas_Log': 'mean', 'Nota':'mean'})

print(agrupadoMedia)

In [41]:
agrupadoMedia

,Aluno,Lab,PP_Delete,Duracao,PP_Paste,Qntd_Linhas,Linhas_Log,Qntd_Linhas_Filtrado,Nota,Submissoes,Testes
0,1085,1031,0.530000,8.000000e+01,0.000000,25.000000,19.000000,9.000000,0.0,0.000000,1.000000
1,1085,1083,0.170000,1.480000e+02,0.005000,12.000000,186.000000,5.500000,0.0,2.000000,6.000000
2,1085,1118,0.172500,9.562500e+02,0.002500,28.000000,871.000000,20.000000,0.0,5.750000,20.500000
3,1085,1265,0.300000,1.864000e+03,0.000000,32.000000,991.000000,20.000000,0.0,17.000000,42.000000
4,1163,1012,0.216429,5.514350e+04,0.007143,2.500000,82.785714,2.071429,0.0,2.428571,1.214286
5,1163,1029,0.211667,4.900946e+05,0.003333,10.666667,416.333333,6.166667,0.0,2.833333,6.416667
6,1163,1051,0.227500,5.140167e+04,0.004167,9.083333,460.750000,6.833333,0.0,2.666667,13.166667
7,1163,1052,0.156667,3.630000e+02,0.003333,8.333333,300.000000,5.666667,0.0,1.000000,3.000000
8,1163,1084,0.204000,1.665611e+05,0.012000,14.500000,415.800000,8.600000,0.0,6.900000,13.300000
9,1163,1089,0.023333,6.829733e+04,0.091667,12.166667,13.333333,8.333333,0.0,1.333333,3.333333


In [42]:
df.to_csv('dadosBrutos.csv', encoding='utf-8', index=False)
agrupado.to_csv('dadosPorLab.csv', encoding='utf-8', index=False)
agrupadoMedia.to_csv('dadosPorLabMean.csv', encoding='utf-8', index=False)

In [43]:
#df.loc[(df['coluna'] == 2) & (df['2'] == 4), 'coluna'] = 'novo valor'